# Introduction: Landmarks

In [1]:
import deltascope as ds
import deltascope.alignment as ut

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize
from scipy.optimize import minimize

import os
import tqdm
import json
import datetime

# Import raw data
The user needs to specify the directories containing the data of interest. Each sample type should have a key which corresponds to the directory path. Additionally, each object should have a list that includes the channels of interest.

In [2]:
# --------------------------------
# -------- User input ------------
# --------------------------------

data = {
    # Specify sample type key
    '30hpf': {
        # Specify path to data directory
        'path': '.\\Data\\30hpf\\Output-02-14-2019',
        # Specify which channels are in the directory and are of interest
        'channels': ['AT','ZRF']
    },
    '28hpf': {
        'path': '.\Data\\28hpf\\Output-02-14-2019-yot-ilastik',
        'channels': ['AT','ZRF']
    },
    '26hpf': {
        'path': '.\Data\\26hpf\\Output-02-14-2019',
        'channels': ['AT','ZRF']
    },
    '24hpf': {
        'path': '.\Data\\24hpf\\Output-02-15-2019',
        'channels': ['AT','ZRF']
    },
    '22hpf': {
        'path': '.\Data\\22hpf\\Output-02-14-2019',
        'channels': ['AT','ZRF']
    }
}

We'll generate a list of pairs of stypes and channels for ease of use.

In [3]:
data_pairs = []
for s in data.keys():
    for c in data[s]['channels']:
        data_pairs.append((s,c))

We can now read in all datafiles specified by the `data` dictionary above.

In [4]:
D = {}
for s in data.keys():
    D[s] = {}
    for c in data[s]['channels']:
        D[s][c] = ds.read_psi_to_dict(data[s]['path'],c)

100%|██████████| 27/27 [00:06<00:00,  2.09it/s]


# Calculate landmark bins

In [5]:
# --------------------------------
# -------- User input ------------
# --------------------------------

# Pick an integer value for bin number 
anum = 30

# Specify the percentiles which will be used to calculate landmarks
percbins = [50]

theta_step = np.pi/4

Calculate landmark bins based on user input parameters and the previously specified control sample.

In [6]:
lm = ds.landmarks(percbins=percbins, rnull=np.nan)
lm.calc_bins(D['28hpf']['AT'], anum, theta_step)

print('Alpha bins')
print(lm.acbins)
print('Theta bins')
print(lm.tbins)

Alpha bins
[-66.16834814 -61.60501378 -57.04167943 -52.47834507 -47.91501072
 -43.35167636 -38.78834201 -34.22500766 -29.6616733  -25.09833895
 -20.53500459 -15.97167024 -11.40833589  -6.84500153  -2.28166718
   2.28166718   6.84500153  11.40833589  15.97167024  20.53500459
  25.09833895  29.6616733   34.22500766  38.78834201  43.35167636
  47.91501072  52.47834507  57.04167943  61.60501378  66.16834814]
Theta bins
[-3.14159265 -2.35619449 -1.57079633 -0.78539816  0.          0.78539816
  1.57079633  2.35619449  3.14159265]


# Calculate landmarks

In [7]:
lmdf = pd.DataFrame()

# Loop through each pair of stype and channels
for s,c in tqdm.tqdm(data_pairs):
    print(s,c)
    # Calculate landmarks for each sample with this data pair
    for k,df in tqdm.tqdm(D[s][c].items()):
        lmdf = lm.calc_perc(df, k, '-'.join([s,c]), lmdf)
        
# Set timestamp for saving data
tstamp = datetime.datetime.now().strftime('%Y-%m-%d')
        
# Save completed landmarks to a csv file
lmdf.to_csv(os.path.join('.\Data',tstamp+'_landmarks.csv'))

# Save landmark bins to json file
bins = {
    'acbins':list(lm.acbins),
    'tbins':list(lm.tbins)
}
with open(os.path.join('.\Data', tstamp+'_landmarks_bins.json'), 'w') as outfile:
    json.dump(bins, outfile)

  0%|          | 0/10 [00:00<?, ?it/s]

30hpf AT



 10%|█         | 1/10 [00:11<01:44, 11.58s/it]

30hpf ZRF



 20%|██        | 2/10 [00:23<01:33, 11.71s/it]

...


 80%|████████  | 8/10 [01:42<00:26, 13.30s/it]

22hpf AT



 90%|█████████ | 9/10 [01:51<00:11, 11.98s/it]

22hpf ZRF



100%|██████████| 10/10 [02:01<00:00, 11.24s/it]
